# Métodos de ensamblado


![fish](img/fish.png)

![title](img/intro.png)

Introducción del artículo *González, S., García, S., Del Ser, J., Rokach, L., & Herrera, F. (2020). A practical tutorial on bagging and boosting based ensembles for machine learning: Algorithms, software tools, performance study, practical perspectives and opportunities. Information Fusion, 64, 205-237.*

Los métodos de ensamblado entrenan múltiples clasificadores (weak learners) para el mismo problema y combinan los resultados (formando un strong learner). El esquema básico sería:

![architecture](img/ensemble_architecture.png)

Extraída del libro *Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.*

Un ensamblado está formado por distintos clasificadores como puede ser un árbol de decisión, una regresión logística u otros tipos de modelos (tanto para el caso de clasificación como el de regresión).

Un ensamblado se construye en dos pasos:
 - generando los modelos base (*weak learners*)
 - y combinándolos después.

¿Coste computacional?
Para seleccionar un único modelo hay que probar varios y elegir bien los parámetros. Esto sería parecido a entrenar distintos modelos base. La combinación final es simple y, por tanto, tiene bajo coste computacional.

¿Cómo combinar los resultados de los modelos?
Media, media ponderada, voto de la mayoría, etc.

## Tipos de ensamblado

Existen distintas taxonomías de ensamblado de modelos pero los tres tipos más conocidos y usados son:
- Bagging
- Boosting
- Stacking

### Stacking

Consiste en combinar distintos modelos a través de otro modelo (un meta-modelo). Este útimo modelo usa como variables los outputs de los modelos iniciales para obtener una predicción final.


![stacking_scheme](img/stacking.png)

Imagen obtenida de [Jason Brownlee - machinelearningmastery](https://machinelearningmastery.com/tour-of-ensemble-learning-algorithms/).

En Python: [StackingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) y [StackingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html).

### Bagging (Bootstrap Aggregation)

Es un método de ensamblado que entrena en paralelo el mismo tipo de modelo. Una de las motivaciones principales de los modelos de ensamblado en paralelo es aprovechar la independencia de los clasificadores para mejorar el error. Sin embargo, al basarse todos los modelos en los mismos datos, se pierde algo de esa independencia. Por ello, se introduce aleatoriedad en los datos mediante el muestreo con *bootstrap*.

Bootstrap consiste en obtener muestras con reemplazamiento del mismo tamaño que la muestra original.

Bagging consiste en entrenar el mismo modelo en distintas muestras bootstrap del mismo conjunto de datos y luego promediar las predicciones de todos ellos puesto que se reduce la varianza.

La predicción final se obtiene como un promedio en el caso de la regresión y por votación en el caso de la clasificación.

*Out-of-bag error*. Como no todas las muestras se usan para entrenar el modelo, se puede estimar el error con dichas muestras.

![bagging_inffus](img/bagging_inffus.png)

*González, S., García, S., Del Ser, J., Rokach, L., & Herrera, F. (2020). A practical tutorial on bagging and boosting based ensembles for machine learning: Algorithms, software tools, performance study, practical perspectives and opportunities. Information Fusion, 64, 205-237.*

### Boosting

Es un método de ensamblado que entrena de forma secuencial el mismo tipo de modelo. Cada modelo tiene en cuenta los errores de los modelos previos a través de pesos que le dan más importancia a aquellas observaciones que se han sido clasificadas de forma incorrecta anteriormente.

Procedimiento general del método Boosting:

![boosting_procedure](img/boosting_procedure.png)

*Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.*

![boosting_inffus](img/boosting_inffus.png)

*González, S., García, S., Del Ser, J., Rokach, L., & Herrera, F. (2020). A practical tutorial on bagging and boosting based ensembles for machine learning: Algorithms, software tools, performance study, practical perspectives and opportunities. Information Fusion, 64, 205-237.*

El algoritmo anterior muestra el procedimiento general de Boosting, para cada algoritmo en concreto hay que definir *Adjust Distribution* y *Combine Outputs*. Dos de los principales algoritmos son:
- AdaBoost.
- Gradient Boosting.

Ejemplos de AdaBoost:

![xor_ada](img/adaboost_xor.png)

*Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.*

![gaussian_ada](img/gaussian_adaboost.png)

*Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.*

## Referencias

- Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.
- González, S., García, S., Del Ser, J., Rokach, L., & Herrera, F. (2020). A practical tutorial on bagging and boosting based ensembles for machine learning: Algorithms, software tools, performance study, practical perspectives and opportunities. Information Fusion, 64, 205-237.
- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). An introduction to statistical learning (Vol. 112, p. 18). New York: springer.
- Hastie, T., Tibshirani, R., Friedman, J. H., & Friedman, J. H. (2009). The elements of statistical learning: data mining, inference, and prediction (Vol. 2, pp. 1-758). New York: springer.

## Ejemplo de Boosting en Python

- Gradient Boosting Machine (GBM)
- Extreme Gradient Boosting (XGBoost). Es una mejora del Gradient Boosting que se centra principalmente en temas de cómputo y de memoria. Aunque también incluye adaptaciones para reducir el overfitting.
- CatBoost. Acepta variables categóricas. [Tutorial en Python](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb)
- HistGradientBoostingClassifier. Más eficiente que GBM en Python cuando n>=10000. También acepta variables categóricas.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
pima = pd.read_csv('pima.csv')
pima.head()

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,class
0,14,175,62,30,0,33.6,0.212,38,1
1,4,146,78,0,0,38.5,0.520,67,1
2,15,136,70,32,110,37.1,0.153,43,1
3,3,107,62,13,48,22.9,0.678,23,1
4,3,169,74,19,125,29.9,0.268,31,1


In [3]:
y = pima['class'].to_numpy()
X = pima.iloc[:,0:-1].to_numpy()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42, stratify = y)

In [5]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
model = AdaBoostClassifier(random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_pred,y_test)

0.7402597402597403

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_pred,y_test)

0.7575757575757576

In [7]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01, use_label_encoder = False,verbosity = 0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_pred,y_test)

0.7792207792207793

In [10]:
!pip install xgboost

In [13]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# categorizamos edad
bins = [0, 30, 40, 50, 60, 70, 150]
group_names = [1,2,3,4,5,6]
pima['age_cat'] = pd.cut(pima[' Age'], bins, labels=group_names)

In [ ]:
pima2 = pima.drop(columns=" Age")

In [ ]:
y2 = pima2['class']
X2 = pima2.drop(columns="class")

In [ ]:
print(X2.dtypes)

In [ ]:
categorical_features_indices = np.where(X2.dtypes == 'category')[0]
print(categorical_features_indices)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2,test_size = 0.3,random_state = 42, stratify = y2)

In [ ]:
from catboost import CatBoostClassifier
model=CatBoostClassifier()
model.fit(X_train2,y_train2,cat_features=categorical_features_indices, verbose = 0)
y_pred = model.predict(X_test2)

In [ ]:
y_pred2 = model.predict(X_test2)
predictions_probs = model.predict_proba(X_test2)
print(y_pred2[:10])
print(predictions_probs[:10])

In [ ]:
# Funcion para calcular accuracy
def compute_accuracy(y_true, y_pred):
    correct_predictions = 0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        if true == predicted:
            correct_predictions += 1
    # compute the accuracy
    accuracy = correct_predictions/len(y_true)
    return accuracy

In [ ]:
compute_accuracy(y_test2, y_pred2)

In [ ]:
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
mod = HistGradientBoostingClassifier(max_bins=255, categorical_features = categorical_features_indices)

In [ ]:
mod.fit(X_train2, y_train2)
y_pred2 = mod.predict(X_test2)


In [ ]:
compute_accuracy(y_test2, y_pred2)